<a href="https://colab.research.google.com/github/aditya05133536/Demand-Forecasting-System-/blob/main/forcast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
from prophet import Prophet

In [ ]:
import pickle

In [ ]:
transactional_data_retail01 = pd.read_csv('/Transactional_data_retail_01.csv')
transactional_data_retail02 = pd.read_csv('/Transactional_data_retail_02.csv')
customer_demographics = pd.read_csv('/CustomerDemographics.csv')
product_info = pd.read_csv('/ProductInfo.csv')


In [ ]:
transactional_data = pd.concat([transactional_data_retail01, transactional_data_retail02])


In [ ]:
def exploratory_data_analysis(data):
    print("Summary statistics:")
    print(data.describe())
    print("\nMissing values:")
    print(data.isnull().sum())

exploratory_data_analysis(transactional_data)


Summary statistics:
            Quantity          Price    Customer ID
count  344319.000000  344318.000000  253263.000000
mean       10.171417       4.886895   15310.133470
std       205.990194     128.024246    1700.931004
min    -74215.000000  -53594.360000   12346.000000
25%         1.000000       1.250000   13897.000000
50%         3.000000       2.250000   15216.000000
75%        10.000000       4.250000   16814.000000
max     74215.000000   16888.020000   18287.000000

Missing values:
Invoice            0
StockCode          0
Quantity           0
Price              1
Customer ID    91056
InvoiceDate        2
dtype: int64


In [ ]:
top_10_stock_codes = transactional_data.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False).head(10)

In [ ]:
transactional_data["revenue"]=transactional_data["Quantity"]*transactional_data["Price"]

In [ ]:
top_10_revenue_products = transactional_data.groupby('StockCode')['revenue'].sum().sort_values(ascending=False).head(10)

In [ ]:
def prepare_data(transactional_data, stock_code):
    # Filter data for the specific stock code
    data = transactional_data[transactional_data['StockCode'] == stock_code].copy()

    # Convert 'InvoiceDate' to datetime
    data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors='coerce')

    # Aggregate the data by date
    data_grouped = data.groupby('InvoiceDate')['Quantity'].sum().reset_index()

    # Rename columns to 'ds' and 'y' as required by Prophet
    data_grouped.rename(columns={'InvoiceDate': 'ds', 'Quantity': 'y'}, inplace=True)

    # Ensure no missing values
    data_grouped = data_grouped.dropna(subset=['ds', 'y'])

    # Ensure correct data types
    data_grouped['ds'] = pd.to_datetime(data_grouped['ds'])
    data_grouped['y'] = data_grouped['y'].astype(float)
    data_grouped = data_grouped.sort_values('ds')

    return data_grouped

In [ ]:
def time_series_analysis(data):
    # Initialize and fit the Prophet model
    model = Prophet()
    model.fit(data)

    # Create future dates
    future = model.make_future_dataframe(periods=15, freq='W')

    # Forecast
    forecast = model.predict(future)

    return model, forecast


In [ ]:
for stock_code in top_10_stock_codes.index:
    data = prepare_data(transactional_data, stock_code)
    model, forecast = time_series_analysis(data)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwwpedtw8/9epllouj.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwwpedtw8/r4a5b1_4.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=40325', 'data', 'file=/tmp/tmpwwpedtw8/9epllouj.json', 'init=/tmp/tmpwwpedtw8/r4a5b1_4.json', 'output', 'file=/tmp/tmpwwpedtw8/prophet_model3cwrvlul/prophet_model-20241003184145.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:41:45 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:41:45 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling yearly seasonalit

In [ ]:
with open(f'model_{stock_code}.pkl', 'wb') as f:
        pickle.dump(model, f)

In [ ]:
forecast.to_csv(f'forecast_{stock_code}.csv', index=False)